In [7]:
from tqdm import tqdm
import numpy as np
import re
import pandas as pd
pd.set_option('display.max_rows', 100)

import requests
from bs4 import BeautifulSoup

from time import sleep
from sqlalchemy import create_engine

from config import ENGINE_LINK

# Parsing

In [8]:
def clean_text(x: str) -> str:
    x = re.sub(r'[\n\t\r]', ' ', x)
    
    return re.sub(r' +', ' ', x).strip()

In [ ]:
for year in range(2011, 2019):
    for country in ['Global', 'USA', 'Europe', 'UK', 'Germany', 'France', 'Japan']:
        print(f'{year} - {country}')
        
        r = requests.get(f'https://www.vgchartz.com/yearly/{year}/{country}/')
        
        soup = BeautifulSoup(r.content)
        
        data = []

        trs = soup.find('table', 'chart').find_all('tr', recursive=False)[1:]

        for tr in trs:
            tds = tr.find_all('td', recursive=False)
            
            first_text = clean_text(tds[1].text)
            platform = re.search(r'\(\w+\)', first_text).group(0)
            try:
                publisher = first_text.split(f'{platform} {platform}')[1]
            except:
                publisher = first_text.split(platform)[1].strip()
            
            try:
                d = {
                    'name' : first_text.split(platform)[0].strip(),
                    'platform' : platform.replace('(', '').replace(')', ''),
                    'publisher' : publisher.split(', ')[0],
                    'genre' : publisher.split(', ')[1],
                    'sales_year' : year,
                    'country' : country,
                    'weeks' : int(clean_text(tds[2].text)),
                    'yearly' : int(clean_text(tds[3].text).replace(',', '')),
                    'total' : int(clean_text(tds[4].text).replace(',', '')),
                    'img' : tr.find('img')['src']
                }
            
            except ValueError:
                continue
            
            data.append(d)
        
        df = pd.DataFrame(data)
        
        with create_engine(ENGINE_LINK).connect() as conn:
            df.to_sql('video_games', con=conn, if_exists='append', index=False)
            
        sleep(1)

# Preprocessing

In [9]:
with create_engine(ENGINE_LINK).connect() as conn:
    df = pd.read_sql(f"select * from video_games where name not in ('', 'Unknown', 'mixi')", conn).drop_duplicates()
    
    
publishers_countries = pd.read_excel('./data/publishers_countries.xlsx')

df = df.merge(publishers_countries, on='publisher', how='inner')

df = df.loc[~df['publisher_country'].isna()].reset_index(drop=True)

df.head(3)

,name,platform,publisher,genre,sales_year,country,weeks,yearly,total,img,publisher_country
0,Call of Duty: Modern Warfare 3,X360,Activision,Shooter,2011,Global,8,12561893,12561893,https://www.vgchartz.com/games/boxart/full_cal...,USA
1,Call of Duty: Modern Warfare 3,PS3,Activision,Shooter,2011,Global,8,10320405,10320405,https://www.vgchartz.com/games/boxart/full_cal...,USA
2,Call of Duty: Black Ops,X360,Activision,Shooter,2011,Global,60,2665288,13186555,https://www.vgchartz.com/games/boxart/full_cal...,USA


In [10]:
def clean_publishers(x: str) -> str:
    try:
        return x.split(')')[1]
    except IndexError:
        return x
    
df['publisher'] = df['publisher'] .map(clean_publishers)

In [11]:
df.loc[df['publisher'].isin(['D3Publisher', 'D3Publisher ']), 'publisher'] = 'D3Publisher'

df.loc[df['publisher'].isin(['Falcom', 'Falcom Corporation']), 'publisher'] = 'Falcom'

df.loc[df['publisher'].isin(['FuRyu Corporation']), 'publisher'] = 'FuRyu'

df.loc[df['publisher'].isin(['Kadokawa Shoten']), 'publisher'] = 'Kadokawa Games'

df.loc[df['publisher'].isin(['Marvelous', 'Marvelous Entertainment', 'Marvelous Interactive']), 'publisher'] = 'Marvelous'

df.loc[df['publisher'].isin(['Microsoft Game Studios', 'Microsoft Studios']), 'publisher'] = 'Microsoft'

df.loc[df['publisher'].isin(['Nippon Ichi Software', 'Nippon Columbia']), 'publisher'] = 'Nippon'

df.loc[df['publisher'].isin(['Sony Computer Entertainment', 'Sony Computer Entertainment America', 'Sony Computer Entertainment Europe', 'Sony Interactive Entertainment']), 'publisher'] = 'Sony'

df.loc[df['publisher'].isin(['Spike', 'Spike Chunsoft']), 'publisher'] = 'Spike'

df.loc[df['publisher'].isin(['Square Enix', 'Square Enix ']), 'publisher'] = 'Square Enix'

In [12]:
df

,name,platform,publisher,genre,sales_year,country,weeks,yearly,total,img,publisher_country
0,Call of Duty: Modern Warfare 3,X360,Activision,Shooter,2011,Global,8,12561893,12561893,https://www.vgchartz.com/games/boxart/full_cal...,USA
1,Call of Duty: Modern Warfare 3,PS3,Activision,Shooter,2011,Global,8,10320405,10320405,https://www.vgchartz.com/games/boxart/full_cal...,USA
2,Call of Duty: Black Ops,X360,Activision,Shooter,2011,Global,60,2665288,13186555,https://www.vgchartz.com/games/boxart/full_cal...,USA
3,Call of Duty: Black Ops,PS3,Activision,Shooter,2011,Global,60,2459689,11165574,https://www.vgchartz.com/games/boxart/full_cal...,USA
4,Skylanders: Spyro's Adventure,Wii,Activision,Action,2011,Global,12,1118799,1118799,https://www.vgchartz.com/games/boxart/full_759...,USA
...,...,...,...,...,...,...,...,...,...,...,...
5222,Red Dead Redemption 2,PS4,Rockstar Games,Action-Adventure,2018,Japan,10,213013,213013,https://www.vgchartz.com/games/boxart/full_267...,USA
5223,The Legend of Heroes: Trails of Cold Steel IV,PS4,Falcom,Role-Playing,2018,Japan,14,129771,129771,https://www.vgchartz.com/games/boxart/full_706...,Japan
5224,Fate/Extella Link,PS4,Marvelous,Action,2018,Japan,30,121569,121569,https://www.vgchartz.com/games/boxart/full_941...,Japan
5225,Fate/Extella Link,PSV,Marvelous,Action,2018,Japan,30,61971,61971,https://www.vgchartz.com/games/boxart/full_908...,Japan


In [13]:
df.to_csv('./data/final_data.csv', index=False)